In [1]:
import oracledb as od
import pandas as pd
import numpy as np
import config
from tqdm import tqdm
import ast
import re

In [2]:
def load_recipe(n =1000):
    od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12") # db connection
    conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],  dsn = config.DB_CONFIG['dsn'])
    exe = conn.cursor()
    exe.execute(f'select * from recipe_table where rownum <= {n}')
    row = exe.fetchall() # row 불러오기
    column_name = exe.description # column 불러오기
    columns=[]
    for i in column_name:
        columns.append(i[0])
    result = pd.DataFrame(row, columns=columns) # row, column을 pandas DataFrame으로 나타내기
    result.rename(mapper=str.lower, axis='columns', inplace=True)
    conn.close()
    return result

In [4]:
raw_data = load_recipe(n = 1000)

In [5]:
raw_data

,recipe_url,recipe_title,recipe_summary1,recipe_summary2,recipe_summary3,recipe_ingredients,recipe_step,recipe_tag,recipe_comment,recipe_view,recipe_category_type,recipe_category_method,recipe_category_situation
0,http://www.10000recipe.com/recipe/6966796,나 순대라고 말 못해요! HACCP인증 제품,1인분,30분 이내,아무나,"{'재료': ['HACCP인증순대90g', 'HACCP인증체다치즈4장', 'HACC...","['#1 HACCP인증 제품입니다', '#2 맛술,요리당,고추장,후추,굴소스 HAC...","['HACCP', '해썹인증제품', '맛있는한끼', '순대요리', '순대말이조림']",1,570,메인반찬,조림,술안주
1,http://www.10000recipe.com/recipe/6851649,시래기 된장국,4인분,30분 이내,아무나,"{'재료': ['신광에코팜시래기1팩(300g)', '된장2T', '멸치1주먹', '...","['#1 먼저 물에 멸치 한주먹과 다시마 1조각을 넣고 육수를 우려내 주세요~!',...","['시래기', '무청', '시래기국', '된장', '시래기 된장국', '국', '반...",0,4449,국/탕,끓이기,일상
2,http://www.10000recipe.com/recipe/6983118,바질크림파스타만들기,3인분,20분 이내,아무나,"{'재료': ['폰타나제노바치즈&바질페스토1병', '통마늘6알', '베이컨3줄', ...",['#1 폰타나 소스를 주로 이용하는 편이예요 베이컨머쉬룸 소스도 맛있더라구요 신랑...,"['폰타나소스로파스타만들기', '바질크림파스타']",0,138,양식,볶음,초스피드
3,http://www.10000recipe.com/recipe/6946145,돼지 수육 삶는법? 보쌈 삶지 말고 찌세요,3인분,120분 이내,초급,"{'수육 위에 얹어줄 재료': ['돼지고기3덩이', '양파1개', '대파또는쪽파원하...",['#1 커피 3스푼과 소금 1스푼을 따뜻한 물 1컵에 녹여주세요. 그리고 찜기 밑...,"['돼지고기', '수육', '보쌈', '보쌈찌기', '수육찌기', '돼지고기요리',...",1,7825,메인반찬,찜,영양식
4,http://www.10000recipe.com/recipe/6844953,브런치 참치에그샐러드만들기,2인분,10분 이내,아무나,"{'재료': ['달걀(삶은거)4개', '참치캔1개', '상추(or양상추)', '게맛...","['#1 달걀을 먼저 삶아 줍니다. 완숙은 7분정도이고 반숙은 4분이면 됩니다.',...","['샐러드', '참치샐러드', '에그샐러드', '참치에그샐러드', '레몬즙 올리브유...",5,10000,샐러드,기타,초스피드
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,http://www.10000recipe.com/recipe/6834022,호박전~호박전 쉽고 빠르게 만드는법~!~,2인분,10분 이내,아무나,"{'재료': ['애호박1개', '천일염1/2큰술', '계란2개', '밀가루1/2컵'...",['#1 우리집 마당에 애호박이 열리질 않아서 사다가 썰어서 말리고 호박전도 한개 ...,"['호박전', '부침이', '초간단']",0,23000,메인반찬,부침,초스피드
996,http://www.10000recipe.com/recipe/4536775,석박지 담그는 법,5인분,2시간 이상,초급,"{'재료': ['무큰것1개', '쪽파3줌'], '무절이기 재료': ['소금2.5큰술...","['#1 무는 사진처럼 어슷썰기 해서 소금 2.5큰술, 설탕 2큰술을 넣고 3시간 ...",[],7,79000,김치/젓갈/장류,절임,일상
997,http://www.10000recipe.com/recipe/6881072,얼큰 소고기 시래기국,5인분,60분 이내,아무나,"{'재료': ['소고기300g', '무청시래기3컵', '대파2개', '고추3개', ...",['#1 시장에서 산 삶은무청시래기를 껍질을 제거해주세요~ 그럼 질기지않더라구요.쉽...,"['얼큰한', '소고기', '무청시래기', '해장']",4,42000,국/탕,끓이기,일상
998,http://www.10000recipe.com/recipe/6908428,중독성 강한 마성의 맛/마약 계란장,3인분,20분 이내,초급,"{'재료': ['계란5개', '다시마육수팩1개', '대파1/2개', '양파1/2개'...","['#1 재료 준비해 주세요~', '#2 물3컵에 다시팩 1개 넣고 10분 정도 끓...","['마약계란', '계란장', '마약계란장', '계란요리', '밑반찬', '달걀장',...",0,4404,밑반찬,끓이기,영양식


In [6]:
def recipe_preprocessing(raw):
    data = raw.loc[raw['recipe_ingredients'].notnull()].copy()  # None 값 제거

    def clean_ingredients(ingredients):
        if ingredients is not None:
            ingredients = ingredients.replace('\\ufeff', '').replace('\\u200b', '')
        return ingredients

    def not_empty_ingredients(row):
        return row['recipe_ingredients'].strip() != '{}' # 결측치 제거

    data = data[data.apply(not_empty_ingredients, axis=1)]
    data["recipe_ingredients"] = data["recipe_ingredients"].apply(clean_ingredients)
    result = data[['recipe_title', 'recipe_ingredients']]

    return result

In [7]:
recipe = recipe_preprocessing(raw_data)
recipe

,recipe_title,recipe_ingredients
0,나 순대라고 말 못해요! HACCP인증 제품,"{'재료': ['HACCP인증순대90g', 'HACCP인증체다치즈4장', 'HACC..."
1,시래기 된장국,"{'재료': ['신광에코팜시래기1팩(300g)', '된장2T', '멸치1주먹', '..."
2,바질크림파스타만들기,"{'재료': ['폰타나제노바치즈&바질페스토1병', '통마늘6알', '베이컨3줄', ..."
3,돼지 수육 삶는법? 보쌈 삶지 말고 찌세요,"{'수육 위에 얹어줄 재료': ['돼지고기3덩이', '양파1개', '대파또는쪽파원하..."
4,브런치 참치에그샐러드만들기,"{'재료': ['달걀(삶은거)4개', '참치캔1개', '상추(or양상추)', '게맛..."
...,...,...
995,호박전~호박전 쉽고 빠르게 만드는법~!~,"{'재료': ['애호박1개', '천일염1/2큰술', '계란2개', '밀가루1/2컵'..."
996,석박지 담그는 법,"{'재료': ['무큰것1개', '쪽파3줌'], '무절이기 재료': ['소금2.5큰술..."
997,얼큰 소고기 시래기국,"{'재료': ['소고기300g', '무청시래기3컵', '대파2개', '고추3개', ..."
998,중독성 강한 마성의 맛/마약 계란장,"{'재료': ['계란5개', '다시마육수팩1개', '대파1/2개', '양파1/2개'..."


In [8]:
#1. 식재료 단위 별로 쪼개기
def split_ingredient(data):
    for i in range(1, 21):
        data.loc[:, f'ingredient{i}'] = None
        data.loc[:, f'quantity{i}'] = None
        data.loc[:, f'unit{i}'] = None
   


    # 패턴과 일치하지 않는 데이터를 저장할 딕셔너리
    non_matching_items = {}
    for idx, row in tqdm(data.iterrows(), total=data.shape[0]): #tqdm으로 진행상황 확인
        if row['recipe_ingredients']:
            ingredients_dict = ast.literal_eval(row["recipe_ingredients"]) #딕셔너리 형태로 저장된 recipe_ingredients 불러오기
            ingredient_count = 1
            for category, items in ingredients_dict.items(): #category : 재료, 양념재료, items: 사과1개, 돼지고기600g
                if items:  # 아이템이 존재하는 경우
                    for item in items:
                        match = re.match(r'([가-힣a-zA-Z]+(\([가-힣]+\))?)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item) # 정규식
                        if match:
                            ingredient, _, quantity, unit = match.groups()
                            
                            data.loc[idx, f'ingredient{ingredient_count}'] = ingredient
                            data.loc[idx, f'quantity{ingredient_count}'] = quantity
                            data.loc[idx, f'unit{ingredient_count}'] = unit

                            ingredient_count += 1
                        else:
                            # 패턴과 일치하지 않는 경우 딕셔너리에 추가
                            non_matching_items[idx] = item
        else:
            pass

    # 패턴과 일치하지 않는 데이터 출력
    for idx, item in non_matching_items.items():
        print(f'Row {idx}: {item}')
    return data

In [9]:
new_recipe = split_ingredient(recipe)
new_recipe

  0%|          | 0/992 [00:00<?, ?it/s]

 51%|█████     | 507/992 [00:00<00:00, 655.17it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_17356\1179294136.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[idx, f'unit{ingredient_count}'] = unit
C:\Users\admin\AppData\Local\Temp\ipykernel_17356\1179294136.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[idx, f'ingredient{ingredient_count}'] = ingredient
C:\Users\admin\AppData\Local\Temp\ipykernel_17356\1179294136.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result 

Row 7: 
Row 34: 3색모둠느타리버섯200g
Row 58: 
Row 71: (새송이)버섯
Row 95: 18cm쉬폰틀1개분
Row 105: 
Row 120: 
Row 345: .냉동꽃게2kg
Row 370: (말탈리아티)파스타면적당량
Row 439: 
Row 502: 3층치즈2개
Row 607: 3층치즈4장
Row 610: 
Row 618: ※밥숟가락기준
Row 625: 
Row 681: 
Row 762: (옵션)쪽파
Row 987: 2배식초1스푼
Row 992: 


,recipe_title,recipe_ingredients,ingredient1,quantity1,unit1,ingredient2,quantity2,unit2,ingredient3,quantity3,...,unit61,ingredient62,quantity62,unit62,ingredient63,quantity63,unit63,ingredient64,quantity64,unit64
0,나 순대라고 말 못해요! HACCP인증 제품,"{'재료': ['HACCP인증순대90g', 'HACCP인증체다치즈4장', 'HACC...",HACCP인증순대,90,g,HACCP인증체다치즈,4,장,HACCP인증사각어묵,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,시래기 된장국,"{'재료': ['신광에코팜시래기1팩(300g)', '된장2T', '멸치1주먹', '...",신광에코팜시래기,1,팩,된장,2,T,멸치,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,바질크림파스타만들기,"{'재료': ['폰타나제노바치즈&바질페스토1병', '통마늘6알', '베이컨3줄', ...",폰타나제노바치즈,,None,통마늘,6,알,베이컨,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,돼지 수육 삶는법? 보쌈 삶지 말고 찌세요,"{'수육 위에 얹어줄 재료': ['돼지고기3덩이', '양파1개', '대파또는쪽파원하...",돼지고기,3,덩이,양파,1,개,대파또는쪽파원하는만큼,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,브런치 참치에그샐러드만들기,"{'재료': ['달걀(삶은거)4개', '참치캔1개', '상추(or양상추)', '게맛...",달걀(삶은거),4,개,참치캔,1,개,상추,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,호박전~호박전 쉽고 빠르게 만드는법~!~,"{'재료': ['애호박1개', '천일염1/2큰술', '계란2개', '밀가루1/2컵'...",애호박,1,개,천일염,1/2,큰술,계란,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,석박지 담그는 법,"{'재료': ['무큰것1개', '쪽파3줌'], '무절이기 재료': ['소금2.5큰술...",무큰것,1,개,쪽파,3,줌,소금,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,얼큰 소고기 시래기국,"{'재료': ['소고기300g', '무청시래기3컵', '대파2개', '고추3개', ...",소고기,300,g,무청시래기,3,컵,대파,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,중독성 강한 마성의 맛/마약 계란장,"{'재료': ['계란5개', '다시마육수팩1개', '대파1/2개', '양파1/2개'...",계란,5,개,다시마육수팩,1,개,대파,1/2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def recipe_food_matrix(data):
    def convert_fraction_to_float(quantity):
        from fractions import Fraction

        try:
            return float(Fraction(quantity))
        except ValueError:
            return None 
    def convert_unit_to_number(unit):
        '''
        단위에 따른 g 수 변환
        '''
        unit_conversion = {
            'g': 1,
            '개': 100,
            '조금' :10
        }
        return unit_conversion.get(unit, 1)
    ingredient_columns = data.filter(like='ingredient').drop(columns=['recipe_ingredients'])
    all_ingredients = [item for sublist in ingredient_columns.values for item in sublist if pd.notna(item)] 
    all_ingredients = set()
    for i in range(1, 21):  
        all_ingredients.update(data[f'ingredient{i}'].dropna().unique())

    recipe_ingredients_df = pd.DataFrame(columns=list(all_ingredients))

    recipe_rows = []
    for idx, row in data.iterrows():
        recipe_data = {ingredient: 0.0 for ingredient in all_ingredients}  # 모든 식재료를 None으로 초기화
        for i in range(1, 21):  
            ingredient = row[f'ingredient{i}']
            quantity = row[f'quantity{i}']
            unit = row[f'unit{i}']
            if pd.notna(ingredient) and pd.notna(quantity):
                quantity_float = convert_fraction_to_float(quantity)
                if quantity_float is not None:
                    unit_number = convert_unit_to_number(unit) if pd.notna(unit) else 1
                    recipe_data[ingredient] = quantity_float * unit_number
        recipe_rows.append(recipe_data)

    # 새로운 데이터프레임 생성 (모든 식재료를 열로 가짐)

    recipe_ingredients_df = pd.concat([pd.DataFrame([row]) for row in recipe_rows], ignore_index=True)
    recipe_ingredients_df = recipe_ingredients_df.astype('float64')
    recipe_ingredients_df = pd.concat([data['recipe_title'], recipe_ingredients_df], axis=1)
    recipe_ingredients_df = recipe_ingredients_df.rename(columns=str.upper)
    
    return recipe_ingredients_df

In [11]:
new_recipe = recipe_food_matrix(new_recipe)
new_recipe

,RECIPE_TITLE,당근소량,쫄면,찹쌀,쌍화탕,김밥김,박력쌀가루,마늘편,코코넛밀크,코코넛채,...,아몬드파우더,허브솔트,파(다진것),홍두깨살,피자치즈(생략가능),번데기캔,배청,조랭이떡,다진파,깨약간
0,나 순대라고 말 못해요! HACCP인증 제품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,시래기 된장국,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,바질크림파스타만들기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,돼지 수육 삶는법? 보쌈 삶지 말고 찌세요,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,브런치 참치에그샐러드만들기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
484,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
675,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# oracle db의 열의 최대갯수 = 1000개
# 자를 필요가 있음
new_recipe1 = new_recipe.iloc[:, 0:999]
new_recipe1 = new_recipe1.astype('str')
new_recipe1


,RECIPE_TITLE,당근소량,쫄면,찹쌀,쌍화탕,김밥김,박력쌀가루,마늘편,코코넛밀크,코코넛채,...,견과류,전복,달걀후라이,쇠고기(우둔살),베이컨적당량,미트볼,녹말물,오렌지,세세멸치또는잔멸치,느타리버섯
0,나 순대라고 말 못해요! HACCP인증 제품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,시래기 된장국,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,바질크림파스타만들기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,돼지 수육 삶는법? 보쌈 삶지 말고 찌세요,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,브런치 참치에그샐러드만들기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
484,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
675,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
new_recipe2 = pd.concat([new_recipe.iloc[:,:1], new_recipe.iloc[:, 999:1998]], axis = 1)
new_recipe2 = new_recipe2.astype('str')
new_recipe2

,RECIPE_TITLE,대파또는쪽파,다시마(대),후추약간약간,유채유,가나슈,블랙올리브,황설탕,생강고추장,아오노리(파래가루),...,제피가루약간,두무,오트밀,고춧잎,시금치나물조금,노랑,피망작은것,홍고추,호박약간,모차렐라치즈
0,나 순대라고 말 못해요! HACCP인증 제품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,시래기 된장국,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,바질크림파스타만들기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,돼지 수육 삶는법? 보쌈 삶지 말고 찌세요,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,브런치 참치에그샐러드만들기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
732,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
838,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
910,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
new_recipe3 = pd.concat([new_recipe.iloc[:,:1], new_recipe.iloc[:, 1998:]], axis = 1)
new_recipe3 = new_recipe3.astype('str')
new_recipe3

,RECIPE_TITLE,돼지고기(부위상관없음),매실엑기스,식빵(또는좋아하는빵),생강조청,보리쌀,조랭이떡(떡볶이떡가능),바닐라아이스크림,양파작은사이즈,멸치,...,깐호두,생강술,소고기샤브샤브용,빵가루약간,알쿠니아황도,삶은메추리알,면육수,오이네개,옥수수캔,파프리카노랑
0,나 순대라고 말 못해요! HACCP인증 제품,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,시래기 된장국,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,바질크림파스타만들기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,돼지 수육 삶는법? 보쌈 삶지 말고 찌세요,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,브런치 참치에그샐러드만들기,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
732,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
838,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
910,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
table_name = 'test3_table'
columns_info = ', '.join([f"{col} VARCHAR2(50)" for col in new_recipe1.columns])
columns_info

'오리스테이크 VARCHAR2(50), 매운고추 VARCHAR2(50), 샌드위치햄 VARCHAR2(50), 살라미 VARCHAR2(50), 오일 VARCHAR2(50), 민트잎약간 VARCHAR2(50), 진간장또는맛간장 VARCHAR2(50), 단호박분말 VARCHAR2(50), 다진두부 VARCHAR2(50), 새우살 VARCHAR2(50), 원당또는설탕 VARCHAR2(50), 콜라비 VARCHAR2(50), 채썬채소 VARCHAR2(50), 고기시즈닝 VARCHAR2(50), 액젖 VARCHAR2(50), 후추가루약간 VARCHAR2(50), 마늘없어도됨 VARCHAR2(50), 쑥갓약간 VARCHAR2(50), 호밀빵 VARCHAR2(50), 각종허브약간 VARCHAR2(50), 육수(물) VARCHAR2(50), 꽃소금작은술 VARCHAR2(50), 새송이버섯 VARCHAR2(50), 무순 VARCHAR2(50), 파스타 VARCHAR2(50), 소불고기 VARCHAR2(50), 노란프리카 VARCHAR2(50), 냉동만두한접시 VARCHAR2(50), 피자치즈작은거 VARCHAR2(50), 영양부추약간 VARCHAR2(50), 홍합살 VARCHAR2(50), 검은콩가루 VARCHAR2(50), 쪽파잎부분 VARCHAR2(50), 초간장약간 VARCHAR2(50), 크러쉬드레드페퍼 VARCHAR2(50), 다크초콜릿 VARCHAR2(50), 갖가지나물잘게자라 VARCHAR2(50), 다시마육수적당량 VARCHAR2(50), 올리브오일약간 VARCHAR2(50), 스팸 VARCHAR2(50), 머스터드소스 VARCHAR2(50), 샐러드채소 VARCHAR2(50), 물엿 VARCHAR2(50), 크림치즈 VARCHAR2(50), 과일쨈 VARCHAR2(50), 밤 VARCHAR2(50), 물넉넉하게 VARCHAR2(50), 이스트슈가약간 VARCHAR2(50), 당근채썰어서 VARCHAR2(50), 이탈리안시즈닝 VARCHAR2(50), 참치 VAR

In [19]:
# oracle 테이블 생성
import config
import oracledb as od

# db connection
od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12")
conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],
                    dsn = config.DB_CONFIG['dsn'])


# 테이블 이름 및 컬럼 정보 추출
table_name = 'recipe_ingre2'
columns_info = ', '.join([f'"{col}" VARCHAR2(300)' for col in new_recipe2.columns])

# 테이블 생성
create_table_sql = f"CREATE TABLE {table_name} ({columns_info})"

exe = conn.cursor()
exe.execute(create_table_sql)
exe.close()

In [20]:
# oracle db에 insert 
def insert_into_oracle(dataframe, table_name, conn):    
    exe = conn.cursor()    

    insert = [tuple(x) for x in dataframe.values]
    exe.executemany(
        f"INSERT INTO {table_name} VALUES ({','.join([':' + str(i+1) for i in range(len(dataframe.columns))])})",
        insert)
    
    conn.commit()
    exe.close()
    
    
insert_into_oracle(new_recipe2, table_name, conn)
conn.close()

In [12]:
def oracleTopd(query):    
    import oracledb as od
    import pandas as pd
    import config

    # db connection
    od.init_oracle_client(lib_dir=r"C:\Program Files\Oracle\instantclient_21_12")
    conn = od.connect(user = config.DB_CONFIG['user'], password = config.DB_CONFIG['password'],
                      dsn = config.DB_CONFIG['dsn'])
    exe = conn.cursor()
    exe.execute(query)
    
    row = exe.fetchall() # row 불러오기

    column_name = exe.description # column 불러오기
    columns=[]
    
    for i in column_name:
        columns.append(i[0])
    
    # row, column을 pandas DataFrame으로 나타내기
    result = pd.DataFrame(row, columns=columns)
    result.rename(mapper=str.lower, axis='columns', inplace=True)
    
    conn.close()
    
    return result

query = 'select A.*, B.* FROM recipe_ingre1 A LEFT JOIN recipe_ingre2 B ON A.recipe_title = B.recipe_title'
result = oracleTopd(query)
result

,recipe_title,라면사리(선택),상추,파프리카파우더(혹은고춧가루),쇠고기(한우),집된장,식용유적당량,중탕한버터,설탕약간,물반컵,...,바닐라에센스,선드라이토마토,고다치즈,손만두,다진쇠고기,루꼴라,만가닥버섯,고추냉이약간,녹말,채썬양파
0,통오징어 리조또 만들기 (찬밥활용),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,황도포크스테이크! 부드러움에 상큼함까지~ (제철과일 복숭아요리),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,한국인의 소울푸드 돼지고기 김치찌개,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,사과 예쁘게 깍기. 가을 겨울 제철 과일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,샘표 사골육수로 떡만두국 만들기 진한 사골곰탕 맛이예요 :),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,[원팬]콩나물불고기 일명 콩불 황금레쉬피~,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,방게장 만들기 / 간장게장 만들기 - 양싸,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999,비빔국수 양념장 새콤 달콤하게 호로록~,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,저렴한가격으로 특별한 명품 반찬 북어양념구이~,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
columns_info = ', '.join([f'"{col}" VARCHAR2(300)' for col in new_recipe2.columns])
columns_info

'"RECIPE_TITLE" VARCHAR2(300), "다진피클" VARCHAR2(300), "밤고구마" VARCHAR2(300), "훈제닭가슴살" VARCHAR2(300), "케첩조금" VARCHAR2(300), "돈까스" VARCHAR2(300), "애플민트잎" VARCHAR2(300), "소금약간" VARCHAR2(300), "국간장(조선간장)" VARCHAR2(300), "콩국물" VARCHAR2(300), "오뚜기간짜장" VARCHAR2(300), "참치다다끼" VARCHAR2(300), "키위" VARCHAR2(300), "와사비약간" VARCHAR2(300), "아보카도메인재료남은것" VARCHAR2(300), "다시마를우려낸물" VARCHAR2(300), "쯔유(부족한간만큼)" VARCHAR2(300), "레몬즙" VARCHAR2(300), "쥬키니호박" VARCHAR2(300), "가쯔오부시약간" VARCHAR2(300), "분당" VARCHAR2(300), "돼지고기등심(다짐육)" VARCHAR2(300), "과일" VARCHAR2(300), "당근채" VARCHAR2(300), "칠리고추" VARCHAR2(300), "쌀가루박력분" VARCHAR2(300), "시판피넛소스" VARCHAR2(300), "쪽파약간" VARCHAR2(300), "가쓰오부시" VARCHAR2(300), "낫또" VARCHAR2(300), "발사믹글레이즈드" VARCHAR2(300), "빵가루적당량" VARCHAR2(300), "완자전반죽" VARCHAR2(300), "소금적정량" VARCHAR2(300), "치킨스톡" VARCHAR2(300), "바닐라엑스트렉" VARCHAR2(300), "물조금" VARCHAR2(300), "케이퍼" VARCHAR2(300), "카레가루" VARCHAR2(300), "양배추채" VARCHAR2(300), "홀그레인머스터드적당량" VARCHAR2(300), "식포" VARCHAR2(300), "중간멸치" VARCHAR2(300)

In [20]:
def nutri_svd(method, df, n): # method = svd라이브러리 선택df = 입력할 테이블, n = 차원수
    if method == 'sklearn':
        import pandas as pd
        import numpy as np
        from sklearn.decomposition import TruncatedSVD

        nutrients_df = df.drop(columns=['RECIPE_TITLE'])
        matrix = nutrients_df.to_numpy()

        svd = TruncatedSVD(n_components=n)
        result = svd.fit_transform(matrix)
        return result

    # scipy
    elif method == 'scipy':
        import numpy as np
        from scipy.sparse.linalg import svds
        from scipy.linalg import svd

        nutrients_df = df.drop(columns=['RECIPE_TITLE'])
        matrix = nutrients_df.to_numpy()
        matrix = matrix.astype(float) 

        num_components = n
        U, Sigma, Vt = svds(matrix, k=num_components)
        matrix_tr = np.dot(np.dot(U,np.diag(Sigma)), Vt)# output of TruncatedSVD
        return U, Sigma, Vt


In [21]:
a,b,c = nutri_svd('scipy', new_recipe, 10)

ValueError: array must not contain infs or NaNs

In [13]:
new_recipe1.columns

Index(['RECIPE_TITLE', '당근소량', '쫄면', '찹쌀', '쌍화탕', '김밥김', '박력쌀가루', '마늘편',
       '코코넛밀크', '코코넛채',
       ...
       '견과류', '전복', '달걀후라이', '쇠고기(우둔살)', '베이컨적당량', '미트볼', '녹말물', '오렌지',
       '세세멸치또는잔멸치', '느타리버섯'],
      dtype='object', length=999)